<a href="https://colab.research.google.com/github/remytr/Black-Scholes-Model/blob/main/ML_Models_for_Derivative_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alternative Models for Option Pricing

Numerical methods such as finite differences, fourier methods, and Monte Carlo simulation is used to value derivatives.

In earlier notebooks, I have explored the use of the Black-Scholes model. There are several limitations to the Black-Scholes model as it is used only for European options, it assumes geometric Brownian motion for the movement of the derivative's price, no-arbitrage.

This notebook will explore the use of a machine learning model to price an option. Will explore a supervised regression-based model.